In [1]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
# N* 2pt
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_N0pi+/avg/N_2pt.h5'
with h5py.File(basePath) as f:
    ab=15
    t=np.array([
        np.mean([f[cfg][cont][:,16,0,ab,0]+1j*f[cfg][cont][:,16,0,ab,1] for ab in [10,15] for cont in ['NP','N0']],axis=0)
        for cfg in f.keys()])
    # t=np.array([
    #     np.mean([f[cfg][cont][:,16,0,ab,0]+1j*f[cfg][cont][:,16,0,ab,1] for ab in [10] for cont in ['NP']],axis=0)
    #     for cfg in f.keys()])

def tFunc(dat):
    t=np.mean(dat,axis=0)
    t=np.real(t)
    t=np.log(t/np.roll(t,-1,axis=0))
    return np.array([t])

(mean,cov,err)=yu.jackknife(t,tFunc,2)

tmin=1; tmax=10

(Nrow, Ncol) = (1, 1)
(Lrow, Lcol) = (4, 5)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False)

for i in range(1):
    tMean=mean[i,tmin:tmax]; tErr=err[i,tmin:tmax]
    irow=0; icol=0
    axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')

plt.grid()

In [3]:
t=lat.getaEN(0)*lat.aInv
print(t)
np.array([0.75,0.8])*lat.aInv

1166.7159450897575


array([1562.56599789, 1666.73706441])

In [4]:
cfgList=[]
basePath=''
filesDic={}
files2ptKey={}
files3ptKey={}

data={}

def loadData():
    f={}
    for key in filesDic.keys():
        f[key]={}
        for file in filesDic[key]:
            f[key][file] = h5py.File(basePath + file + '.h5','r')

    for key in files2ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for proj2 in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][proj2]={}
                        for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                            data[key][file][projA][projB][proj2][cont]=np.array([
                                f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                                ])

    for key in files3ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for proj2 in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][proj2]={}
                        for cont in tF[projAStr][projBStr][proj2][cfgList[0]].keys():
                            temp_sgn=1
                            if file in ['B','W','Z']: # for source pion sign missed by PLEGMA
                                temp_sgn=-1j 
                            data[key][file][projA][projB][proj2][cont]=np.array([
                                f[key][file][projAStr][projBStr][proj2][cfg][cont][()] for cfg in cfgList
                                ])*temp_sgn

    for key in filesDic.keys():
        for file in filesDic[key]:
            f[key][file].close()


basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_N0pi+/proj/'
with h5py.File(basePath+'N_2pt.h5') as f:
    cfgList=list(yu.deepKey(f,3).keys())

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJN/proj/'
with h5py.File(basePath+'dt14_protonup.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))
with h5py.File(basePath+'dt14_protondn.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))

print(len(cfgList))


basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_N0pi+/proj/'
filesDic={}
files2ptKey=['NN','NNpi','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
files3ptKey=['NJNpi']
filesDic['NJNpi']=['B','W','Z']
loadData()

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/NJN/proj/'
filesDic={}
files2ptKey=[]
files3ptKey=['NJN']
filesDic['NJN']=['dt10_protonup','dt10_protondn','dt12_protonup','dt12_protondn','dt14_protonup','dt14_protondn']
loadData()

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/proj/'
filesDic={}
files2ptKey=['NNpi']
filesDic['NNpi']=['N_pi0Loop']
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['N_IL']
filesDic['NJNpi']=['T_IL','N_pi0Loop_IL']
loadData()

data['others']={}
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/avg/insertLoop.h5'
with h5py.File(basePath) as f:
    data['others']['u+d_id_vev']=np.array([f[cfg]['u+d_id_vev'][()] for cfg in cfgList])
    data['others']['u-d_g5_vev']=np.array([f[cfg]['u-d_g5_vev'][()] for cfg in cfgList])
basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/QuarkLoops_pi0_insertion/avg/pi0Loop.h5'
with h5py.File(basePath) as f:
    data['others']['pi0Loop_g5_vev']=np.array([f[cfg]['pi0Loop_g5_vev'][()] for cfg in cfgList])


308


In [5]:
projList={}
projList['N']=[
    ((0,0,0),'a'),\
    ((0,0,1),'a'),((0,0,-1),'a'),((1,0,0),'a'),\
    ((0,1,1),'a'),\
    ((1,1,1),'a')
]
projList['Npi']=[
    ((0,0,0),('N0\\pi0','G1u_a')),\
    ((0,0,0),('N0\\pi0','a')),\
    ((0,0,0),('N1\\pi1','a')),((0,0,0),('N2\\pi2','a')),((0,0,0),('N3\\pi3','a')),\
    ((0,0,1),('N1\\pi0','a')),((0,0,1),('N0\\pi1','a')),((0,0,1),('N2\\pi1','a')),((0,0,1),('N2\\pi1','b')),\
        ((0,0,-1),('N1\\pi0','a')),((1,0,0),('N1\\pi0','a')),\
    ((0,1,1),('N2\\pi0','a')),((0,1,1),('N1\\pi1','a')),((0,1,1),('N1\\pi1','b')),((0,1,1),('N0\\pi2','a')),\
    ((1,1,1),('N3\\pi0','a')),((1,1,1),('N2\\pi1','a')),((1,1,1),('N2\\pi1','b')),((1,1,1),('N1\\pi2','a')),((1,1,1),('N1\\pi2','b')),((1,1,1),('N0\\pi3','a'))
]

caseDic={} # case[p_tot] = [ [proj of N], [projs of Npi] ]
caseDic[(0,0,0)]=[[((0,0,0),'a')],[((0,0,0),('N0\\pi0','G1u_a')),((0,0,0),('N0\\pi0','a')),((0,0,0),('N1\\pi1','a')),((0,0,0),('N2\\pi2','a')),((0,0,0),('N3\\pi3','a'))]]
caseDic[(0,0,1)]=[[((0,0,1),'a')],[((0,0,1),('N1\\pi0','a')),((0,0,1),('N0\\pi1','a')),((0,0,1),('N2\\pi1','a')),((0,0,1),('N2\\pi1','b'))]]
caseDic[(0,1,1)]=[[((0,1,1),'a')],[((0,1,1),('N2\\pi0','a')),((0,1,1),('N1\\pi1','a')),((0,1,1),('N1\\pi1','b')),((0,1,1),('N0\\pi2','a'))]]
caseDic[(1,1,1)]=[[((1,1,1),'a')],[((1,1,1),('N3\\pi0','a')),((1,1,1),('N2\\pi1','a')),((1,1,1),('N2\\pi1','b')),((1,1,1),('N1\\pi2','a')),((1,1,1),('N1\\pi2','b')),((1,1,1),('N0\\pi3','a'))]]

def getDat2pt(projA,projB):
    if projA in projList['N'] and projB in projList['N']:
        return data['NN']['N_2pt'][projA][projB]['P0']['NP']
    elif projA in projList['N']:
        return 1j*np.sum([data['NNpi']['T'][projA][projB]['P0'][cont] for cont in ['Tseq'+str(i) for i in range(11,15)]],axis=0)
    elif projB in projList['N']:
        return np.conj(getDat2pt(projB,projA))
    else:
        diagList=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
        contDic={'B_2pt':['B'+str(i) for i in range(13,17)],'W_2pt':['W'+str(i) for i in range(25,29)],'Z_2pt':['Z'+str(i) for i in range(5,7)],'M_correct_2pt':['MN0PP']}
        return np.sum([
        np.sum([data['NpiNpi'][diag][projA][projB]['P0'][cont] for cont in contDic[diag]],axis=0)
        for diag in diagList],axis=0)
    
def getDat2ptMat(p_tot,N_op=None):
    projs=(caseDic[p_tot][0]+caseDic[p_tot][1])[:N_op]
    t=np.transpose([[getDat2pt(projA,projB) for projB in projs] for projA in projs],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(Gc,iso,proj,tf,projA,projB):
    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]
    if projA in projList['N'] and projB in projList['N']:
        return data['NJN']['dt'+str(tf)+'_protonup'][projA][projB][proj]['MprotonUp'][:,1:tf,i_Gc]+sgn_iso*data['NJN']['dt'+str(tf)+'_protondn'][projA][projB][proj]['MprotonDn'][:,1:tf,i_Gc]
        # return data['NJN']['protonup'+str(tf)][projA][projB][proj]['MprotonUp'][:,1:tf,i_Gc]+sgn_iso*data['NJN']['protondn'+str(tf)][projA][projB][proj]['MprotonDn'][:,1:tf,i_Gc]
    elif projA in projList['N']:
        contDic = {'B': ['B'+str(i) for i in range(9, 13)], 'Wu': ['W'+str(i) for i in range(17, 21)],
            'Wd': ['W'+str(i) for i in range(21, 25)], 'Z': ['Z'+str(i) for i in range(11, 15)]}
        return np.sum([np.sum([data['NJNpi'][diag[0]][projA][projB][proj][cont+'_deltat_'+str(tf)][:,1:tf,i_Gc] for cont in contDic[diag]],axis=0)
         for diag in ['B','Wu']],axis=0) + sgn_iso * np.sum([np.sum([data['NJNpi'][diag[0]][projA][projB][proj][cont+'_deltat_'+str(tf)][:,1:tf,i_Gc] for cont in contDic[diag]],axis=0)
         for diag in ['Wd','Z']],axis=0)
    elif projB in projList['N']:
        if projB[0] != (0,0,0):
            return np.zeros([len(cfgList),tf-1])
        return np.conj(getDat3pt(Gc,iso,proj,tf,projB,projA)[:,::-1])
    else:
        return np.zeros([len(cfgList),tf-1])
    
def getDat3ptMat(Gc,iso,proj,pc,tf,N_opA=None,N_opB=None):
    projsA=(caseDic[(0,0,0)][0]+caseDic[(0,0,0)][1])[:N_opA]
    projsB=(caseDic[pc][0]+caseDic[pc][1])[:N_opB]
    t=np.transpose([[getDat3pt(Gc,iso,proj,tf,projA,projB) for projB in projsB] for projA in projsA],[2,3,0,1])
    return t

tfList=[10,12,14]
def getDat(Gc,iso,proj,pc,N_opA=None,N_opB=None):
    proj2=proj
    if pc==(0,0,0) and proj=='Pzt':
        proj2='Pz'
    dat2ptA=getDat2ptMat((0,0,0),N_opA)
    dat2ptB=getDat2ptMat(pc,N_opB)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(Gc,iso,proj2,pc,tf,N_opA,N_opB)
    t=np.array([
        np.array([dat2ptA[cfg],dat2ptB[cfg]]+[dat3pt[tf][cfg] for tf in tfList],dtype=object)
    for cfg in range(len(cfgList))], dtype=object)
    return t

In [ ]:
N_op=5
t=getDat2ptMat((0,0,0),N_op)

tRef=5

def tFunc(dat):
    t=np.mean(dat,axis=0)
    # (eVals,eVecs)=yu.GEVP(t,tRef)
    eVals=np.transpose([t[:,i,i] for i in range(N_op)])
    eVals=np.real(eVals)
    En=np.log(eVals/np.roll(eVals,-1,axis=0))
    return En.T

(mean,cov,err)=yu.jackknife(t,tFunc,2)

tmin=1; tmax=12
# print(mean)
# print(err)

(Nrow, Ncol) = (1, 1)
(Lrow, Lcol) = (4*2, 5*2)
fig, axs = plt.subplots(Nrow, Ncol, figsize=(Lcol*Ncol, Lrow*Nrow), squeeze=False)

for i in range(N_op):
    tMean=mean[i,tmin:tmax]; tErr=err[i,tmin:tmax]
    irow=0; icol=0
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange'}
    axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color=colorDic[i],fmt='s',elinewidth=2,capsize=6,mfc='white')

In [ ]:
tRef=1
tmin=1;tmax=15

pdfFlag=False
def plot():
    rows=[((0,0,0),6),
        ((0,0,1),5),
        ((0,1,1),5),
        ((1,1,1),7),]

    (Nrow, Ncol) = (len(rows), 3)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
    fig.suptitle('2pt-GEVP')
    yu.addRowHeader(axs,['p_tot=\n{}'.format(row[0]) for row in rows])
    yu.addColHeader(axs,['E-values','E-vectors','Ratio'])
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}

    for irow in range(Nrow):
        (p_tot,N_op)=rows[irow]
        dat=getDat2ptMat(p_tot,N_op)

        #============
        icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T

        (mean,cov,err)=yu.jackknife(dat,tFunc,2)
        
        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,color=colorDic[i],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,2))
        axs[irow,icol].set_ylim(.4,1.4)
        axs[irow,icol].grid()

        #============
        icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            t=eVecs[:,:,0].T
            return np.abs(t)

        (mean,cov,err)=yu.jackknife(dat,tFunc,2)

        # print(mean[0,10]/mean[1,10])

        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,color=colorDic[i],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,2))
        axs[irow,icol].grid()

        #============
        icol=2
        mean0=mean[0,tmin:tmax]

        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax]/mean0;tErr=err[i,tmin:ttmax]/mean0
            print(irow,i,tMean[10],tErr[10])
            axs[irow,icol].errorbar(np.arange(tmin,ttmax),tMean,tErr,color=colorDic[i],fmt='s',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,2))
        axs[irow,icol].set_ylim(1e-4,1.1)
        axs[irow,icol].set_yscale('log')
        axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])
        axs[irow,icol].grid()

    if pdfFlag:
        plt.close() 
    return fig

pdfPath='fig/'+ens+'/2ptGEVP.pdf'
with PdfPages(pdfPath) as pdf:
    pdfFlag=True
    pdf.savefig(plot())
    pdfFlag=False


In [8]:
tRef=1
tmin=1;tmax=15

pdfFlag=False
def plot(Gc,iso,proj,preFactor,ylimRatio):
    rows=[((0,0,0),(2,2),(6,6)),
        ((0,0,1),(2,2),(4,5)),
        ((0,1,1),(2,2),(4,3)),
        ((1,1,1),(2,2),(4,3)),]

    (Nrow, Ncol) = (len(rows), 5)
    fig, axs = plt.subplots(Nrow, Ncol, figsize=(5*Ncol, 4*Nrow), squeeze=False)
    fig.suptitle('2pt3pt-GEVP: (Gc,iso,proj,preFactor)='+str((Gc,iso,proj,preFactor)))
    yu.addRowHeader(axs,['pc=\n{}'.format(row[0]) for row in rows])
    yu.addColHeader(axs,['3pt','3pt/2pt: only NJN operator','3pt/2pt: few operators','3pt/2pt: more operators','3pt/2pt: comparison'])

    for irow in range(Nrow):
        (pc,(N_opA,N_opB),(N_opA2,N_opB2))=rows[irow]
        dat=getDat(Gc,iso,proj,pc,N_opA,N_opB)

        #============
        icol=0
        # for i in range(N_opA):
        i=0
        for j in range(N_opB):
            def tFunc(dat):
                t=np.mean(dat,axis=0)
                t=np.array([np.real(preFactor*t[ind_tf+2][:,i,j]) for ind_tf in range(len(tfList))],dtype=object)
                return t
        
            (mean,cov,err)=yu.jackknife(dat,tFunc,2)

            colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
            fmtDic={10:'s',12:'d',14:'o'}

            for i_tf in range(len(tfList)):
                tf=tfList[i_tf]
                tMean=mean[i_tf];tErr=err[i_tf]
                axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                    fmt=fmtDic[tf],color=colorDic[(i,j)],elinewidth=2,capsize=6,mfc='white')
            axs[irow,icol].set_xticks(range(-6,6,2))

        #============
        # for (t_N_opA,t_N_opB) in [(1,1),(N_opA,N_opB),(None,None)]:
        for (t_N_opA,t_N_opB) in [(1,1),(N_opA,N_opB),(N_opA2,N_opB2)]:
            dat=getDat(Gc,iso,proj,pc,t_N_opA,t_N_opB)
            def tFunc(dat):
                t=np.mean(dat,axis=0)
                (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
                (eValsb,eVecsb)=yu.GEVP(t[1],tRef)
                def getRes(ind_tf):
                    tf=tfList[ind_tf]
                    eVeca=eVecsa[tf,:,0]
                    eVecb=eVecsb[tf,:,0]
                    denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                    np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                    np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                    nom=np.conj(eVeca).T@t[2+ind_tf]@eVecb
                    return nom/denom
                t=np.array([np.real(preFactor*
                    getRes(ind_tf)
                    ) for ind_tf in range(len(tfList))],dtype=object)
                return t
            
            colorDic={(1,1):'r',(N_opA,N_opB):'g',(N_opA2,N_opB2):'b'}
            fmtDic={10:'s',12:'d',14:'o'}
            icol1Dic={(1,1):1,(N_opA,N_opB):2,(N_opA2,N_opB2):3}
            icol2=4
            for icol in [icol1Dic[(t_N_opA,t_N_opB)],icol2]:
                (mean,cov,err)=yu.jackknife(dat,tFunc,2)
                for i_tf in range(len(tfList)):
                    tf=tfList[i_tf]
                    tMean=mean[i_tf];tErr=err[i_tf]
                    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                        fmt=fmtDic[tf],color=colorDic[(t_N_opA,t_N_opB)],elinewidth=2,capsize=6,mfc='white')
                axs[irow,icol].set_xticks(range(-6,6,2))
                axs[irow,icol].set_ylim(ylimRatio)

    if pdfFlag:
        plt.close()
    return fig

pdfPath='fig/'+ens+'/2pt3ptGEVP.pdf'
with PdfPages(pdfPath) as pdf:
    pdfFlag=True

    pdf.savefig(plot('id','+','P0',1,(0,17)))
    pdf.savefig(plot('id','-','P0',1,(0,3.8)))

    pdf.savefig(plot('gt','+','P0',1,(0,5.3)))
    pdf.savefig(plot('gt','-','P0',1,(0,2)))

    pdf.savefig(plot('g5','+','Pzt',1,(-0.5,3)))
    pdf.savefig(plot('g5','-','Pzt',1,(-0.5,5)))

    pdf.savefig(plot('g5gz','+','Pzt',-1j,(-1,0)))
    pdf.savefig(plot('g5gz','-','Pzt',-1j,(-2,0)))

    pdfFlag=False


In [ ]:
with h5py.File('/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pi0InsertionPost/proj/M.h5') as f:
    tF=f["((0, 0, 0), 'a')"][ "((0, 0, 0), ('N1\\\\pi1', 'a'))"]["P0"]
    cfgs=tF.keys()
    Gc=5
    dat=np.array([tF[cfg]['up_deltat_10'][:,Gc]-0*tF[cfg]['dn_deltat_10'][:,Gc] for cfg in tF.keys()])
    dat=np.real(dat)
    (mean,cov,err)=yu.jackknife(dat)
    print(mean)
    print(err)

In [ ]:
Gc=5;ab=0;sgn=1;ri=np.real
Gc=0;ab=0;sgn=-1;ri=np.imag


print('up')
with h5py.File('/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pi0InsertionPost/avg/M.h5') as f:
    dat=np.array([f[cfg]['pi2=0_0_0/up_deltat_10'][:,16,Gc,ab,0]+1j*f[cfg]['pi2=0_0_0/up_deltat_10'][:,16,Gc,ab,1] for cfg in f.keys()])
    dat=ri(dat)
    (mean,cov,err)=yu.jackknife(dat)
    print(mean)
    print(err)
print('dn')
with h5py.File('/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pi0InsertionPost/avg/M.h5') as f:
    dat=np.array([f[cfg]['pi2=0_0_0/dn_deltat_10'][:,16,Gc,ab,0]+1j*f[cfg]['pi2=0_0_0/dn_deltat_10'][:,16,Gc,ab,1] for cfg in f.keys()])
    dat=ri(dat)
    (mean,cov,err)=yu.jackknife(dat)
    print(mean)
    print(err)
print('up+/-dn')
with h5py.File('/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA211.53.24/pi0InsertionPost/avg/M.h5') as f:
    dat=np.array([f[cfg]['pi2=0_0_0/up_deltat_10'][:,16,Gc,ab,0]+1j*f[cfg]['pi2=0_0_0/up_deltat_10'][:,16,Gc,ab,1]\
        +sgn*(f[cfg]['pi2=0_0_0/dn_deltat_10'][:,16,Gc,ab,0]+1j*f[cfg]['pi2=0_0_0/dn_deltat_10'][:,16,Gc,ab,1])\
              for cfg in f.keys()])
    dat=ri(dat)
    (mean,cov,err)=yu.jackknife(dat)
    print(mean)
    print(err)